In [1]:
import os
from datasets import load_dataset, concatenate_datasets
from tqdm.auto import tqdm

files = [
    os.path.join("./data/", f) for f in os.listdir("./data/") if f.endswith(".json")
]

In [2]:
def preprocess(example):
    data = {}
    data['startTime'] = example['startTime']['$date']
    data['geolocationEvents'] = [{
        'accuracy': event['accuracy'],
        'altitude': event['altitude'],
        'heading': event['heading'],
        'is_moving': event['is_moving'],
        'latitude': event['latitude'],
        'longitude': event['longitude'],
        'speed': event['speed'],
    } for event in example["geolocationEvents"]]

    return data

In [3]:
dataset = load_dataset("json", data_files=files[0], split='train')

x = [d for d in dataset if d['overriddenDistances'] is not None]

In [5]:
x[2]

{'_id': {'$oid': '6402e90e8a7bfd2bf0ac5766'},
 'clientId': 'train_1677477767559',
 'userId': 'u_c21be8acc13d4df4a2601ff02a2b235b',
 'nickname': 'matilda',
 'territoryId': 'L',
 'multimodalId': 'multimodal_1677477767559',
 'freeTrackingTransport': 'train',
 'geolocationEvents': [{'accuracy': 100,
   'activity_confidence': 33,
   'activity_type': 'still',
   'altitude': 322.0,
   'battery_is_charging': True,
   'battery_level': 0.25,
   'created_at': {'$date': '2023-03-04T06:45:34.279Z'},
   'device_model': 'iPhone',
   'geocoding': [9.290064915539658, 46.015983639595945],
   'heading': -1.0,
   'is_moving': False,
   'latitude': 46.015983639595945,
   'longitude': 9.290064915539658,
   'multimodalId': 'multimodal_1677477767559',
   'recorded_at': {'$date': '2023-03-03T21:37:50.989Z'},
   'sharedTravelId': None,
   'speed': -1.0,
   'travelId': 'train_1677477767559',
   'userId': 'u_c21be8acc13d4df4a2601ff02a2b235b',
   'uuid': 'B97E2A06-7FF4-45EC-B831-3562B9C39C13'},
  {'accuracy': 5,
 

In [ ]:
dataset = load_dataset("json", data_files=files[0], split='train').select_columns(['userId', 'geolocationEvents', 'startTime'])
dataset = dataset.map(preprocess, num_proc=min(os.cpu_count(), len(dataset)))

for f in tqdm(files[1:]):
    d = load_dataset("json", data_files=f, split='train').select_columns(['userId', 'geolocationEvents', 'startTime'])
    d = d.map(preprocess, num_proc=min(os.cpu_count(), len(d)))
    dataset = concatenate_datasets([dataset, d])

dataset